OpenAI recently released Distillation which allows to leverage the outputs of a (large) model to fine-tune another (smaller) model. This can significantly reduce the price and the latency for specific tasks as you move to a smaller model. In this cookbook we'll look at a dataset, distill the output of gpt-4o to gpt-4o-mini and show how we can get significantly better results than on a generic, non-distilled, 4o-mini.

We'll also leverage Structured Outputs for a classification problem using a list of enum. We'll see how fine-tuned model can benefit from structured output and how it will impact the performance. We'll show that Structured Ouputs work with all of those models, including the distilled one.

We'll first analyze the dataset, get the output of both 4o and 4o mini, highlighting the difference in performance of both models, then proceed to the distillation and analyze the performance of this distilled model.

Leveraging model distillation to fine-tune a model



In [ ]:
! pip install openai tiktoken numpy pandas tqdm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
open_api_key=userdata.get('OPENAI_API_KEY')

In [ ]:
import openai
import json
import tiktoken
from tqdm import tqdm
from openai import OpenAI
import numpy as np
import concurrent.futures
import pandas as pd

client = OpenAI(api_key=open_api_key)

**Loading and understanding the dataset**

https://www.kaggle.com/datasets/zynicide/wine-reviews

This dataset has a large number of rows and you're free to run this cookbook on the whole data, but as a biaised french wine-lover, I'll narrow down the dataset to only French wine to focus on less rows and grape varieties.

We're looking at a classification problem where we'd like to guess the grape variety based on all other criterias available, including description, subregion and province that we'll include in the prompt. It gives a lot of information to the model, you're free to also remove some information that can help significantly the model such as the region in which it was produced to see if it does a good job at finding the grape.

Let's filter the grape varieties that have less than 5 occurences in reviews.

Let's proceed with a subset of 500 random rows from this dataset.

In [ ]:
df = pd.read_csv('winemag-data-130k-v2.csv', lineterminator='\n', index_col=0, escapechar='\\')


In [ ]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
df_france = df[df['country'] == 'France']

# Let's also filter out wines that have less than 5 references with their grape variety – even though we'd like to find those
# they're outliers that we don't want to optimize for that would make our enum list be too long
# and they could also add noise for the rest of the dataset on which we'd like to guess, eventually reducing our accuracy.

varieties_less_than_five_list = df_france['variety'].value_counts()[df_france['variety'].value_counts() < 5].index.tolist()
df_france = df_france[~df_france['variety'].isin(varieties_less_than_five_list)]

df_france.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
11,France,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer
30,France,Red cherry fruit comes laced with light tannin...,Nouveau,86,NaN,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Domaine de la Madone 2012 Nouveau (Beaujolais...,Gamay,Domaine de la Madone
42,France,"This is a festive wine, with soft, ripe fruit ...",Nouveau,86,9.0,Beaujolais,Beaujolais,NaN,Roger Voss,@vossroger,Henry Fessy 2012 Nouveau (Beaujolais),Gamay,Henry Fessy


In [ ]:
df_france.size

112736

In [ ]:
df_france_subset = df_france.sample(n=500)
df_france_subset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
50564,France,"This complex, spicy-herbal wine is also well-f...",Les Perrottes,90,30.0,Rhône Valley,Costières de Nîmes,NaN,Joe Czerwinski,@JoeCz,Domaine de Poulvarel 2009 Les Perrottes Red (C...,Rhône-style Red Blend,Domaine de Poulvarel
8335,France,"This is an open, soft, warm wine. It has some ...",NaN,87,20.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Wolfberger 2012 Pinot Gris (Alsace),Pinot Gris,Wolfberger
13922,France,"This wine is quite firm, but with plenty of bl...",Santenots Premier Cru,91,145.0,Burgundy,Volnay,NaN,Roger Voss,@vossroger,Domaine Jacques Prieur 2014 Santenots Premier ...,Pinot Noir,Domaine Jacques Prieur
23415,France,"Cut with sulfur, this is a wine whose bitterne...",NaN,82,13.0,Loire Valley,Rosé d'Anjou,NaN,Roger Voss,@vossroger,Bougrier 2013 Rosé d'Anjou,Rosé,Bougrier
16105,France,"86–88. Barrel sample. A hard-edged wine, the p...",Barrel Sample,87,NaN,Bordeaux,Médoc,NaN,Roger Voss,@vossroger,Château la Tour de By 2012 Barrel Sample (Médoc),Bordeaux-style Red Blend,Château la Tour de By


Let's retrieve all grape varieties to include them in the prompt and in our structured outputs enum list.



In [ ]:
varieties = np.array(df_france['variety'].unique()).astype('str')
varieties

array(['Gewürztraminer', 'Pinot Gris', 'Gamay',
       'Bordeaux-style White Blend', 'Champagne Blend', 'Chardonnay',
       'Petit Manseng', 'Riesling', 'White Blend', 'Pinot Blanc',
       'Alsace white blend', 'Bordeaux-style Red Blend', 'Malbec',
       'Tannat-Cabernet', 'Rhône-style Red Blend', 'Savagnin',
       'Pinot Noir', 'Rosé', 'Melon', 'Rhône-style White Blend',
       'Pinot Noir-Gamay', 'Colombard', 'Chenin Blanc', 'Sylvaner',
       'Sauvignon Blanc', 'Red Blend', 'Chenin Blanc-Chardonnay',
       'Cabernet Sauvignon', 'Cabernet Franc', 'Syrah', 'Sparkling Blend',
       'Duras', 'Provence red blend', 'Tannat', 'Merlot', 'Malbec-Merlot',
       'Muscat', 'Viognier', 'Picpoul', 'Provence white blend',
       'Mondeuse', 'Grenache-Syrah', 'G-S-M', 'Vermentino', 'Marsanne',
       'Gros and Petit Manseng', 'Jacquère', 'Negrette', 'Grenache',
       'Gros Manseng', 'Aligoté', 'Syrah-Grenache'], dtype='<U26')

**Generating the prompt**

Let's build out a function to generate our prompt and try it for the first wine of our list.



In [ ]:
def generate_prompt(row, varieties):
    # Format the varieties list as a comma-separated string
    variety_list = ', '.join(varieties)

    prompt = f"""
    Based on this wine review, guess the grape variety:
    This wine is produced by {row['winery']} in the {row['province']} region of {row['country']}.
    It was grown in {row['region_1']}. It is described as: "{row['description']}".
    The wine has been reviewed by {row['taster_name']} and received {row['points']} points.
    The price is {row['price']}.

    Here is a list of possible grape varieties to choose from: {variety_list}.

    What is the likely grape variety? Answer only with the grape variety name or blend from the list.
    """
    return prompt

# Example usage with a specific row
prompt = generate_prompt(df_france.iloc[0], varieties)
prompt

'\n    Based on this wine review, guess the grape variety:\n    This wine is produced by Trimbach in the Alsace region of France.\n    It was grown in Alsace. It is described as: "This dry and restrained wine offers spice in profusion. Balanced with acidity and a firm texture, it\'s very much for food.".\n    The wine has been reviewed by Roger Voss and received 87 points.\n    The price is 24.0.\n\n    Here is a list of possible grape varieties to choose from: Gewürztraminer, Pinot Gris, Gamay, Bordeaux-style White Blend, Champagne Blend, Chardonnay, Petit Manseng, Riesling, White Blend, Pinot Blanc, Alsace white blend, Bordeaux-style Red Blend, Malbec, Tannat-Cabernet, Rhône-style Red Blend, Savagnin, Pinot Noir, Rosé, Melon, Rhône-style White Blend, Pinot Noir-Gamay, Colombard, Chenin Blanc, Sylvaner, Sauvignon Blanc, Red Blend, Chenin Blanc-Chardonnay, Cabernet Sauvignon, Cabernet Franc, Syrah, Sparkling Blend, Duras, Provence red blend, Tannat, Merlot, Malbec-Merlot, Muscat, Viogn

To get a understanding of the cost before running the queries, you can leverage tiktoken to understand the number of tokens we'll send and the cost associated to run this. This will only give you an estimate for to run the completions, not the fine-tuning process (used later in this notebook when running the distillation), which depends on other factors such as the number of epochs, training set etc.



In [ ]:
# Load encoding for the GPT-4 model
enc = tiktoken.encoding_for_model("gpt-4o")

# Initialize a variable to store the total number of tokens
total_tokens = 0

for index, row in df_france_subset.iterrows():
    prompt = generate_prompt(row, varieties)

    # Tokenize the input text and count tokens
    tokens = enc.encode(prompt)
    token_count = len(tokens)

    # Add the token count to the total
    total_tokens += token_count

print(f"Total number of tokens in the dataset: {total_tokens}")
print(f"Total number of prompts: {len(df_france_subset)}")

Total number of tokens in the dataset: 179983
Total number of prompts: 500


In [ ]:
# outputing cost in $ as of 2024/10/16

gpt4o_token_price = 2.50 / 1_000_000  # $2.50 per 1M tokens
gpt4o_mini_token_price = 0.150 / 1_000_000  # $0.15 per 1M tokens

total_gpt4o_cost = gpt4o_token_price*total_tokens
total_gpt4o_mini_cost = gpt4o_mini_token_price*total_tokens

print(total_gpt4o_cost)
print(total_gpt4o_mini_cost)

0.4499575
0.02699745


**Preparing functions to Store Completions**


As we're looking at a limited list of response (enumerate list of grape varieties), let's leverage structured outputs so we make sure the model will answer from this list. This also allows us to compare the model's answer directly with the grape variety and have a deterministic answer (compared to a model that could answer "I think the grape is Pinot Noir" instead of just "Pinot noir"), on top of improving the performance to avoid grape varieties not in our dataset.

In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "grape-variety",
        "schema": {
            "type": "object",
            "properties": {
                "variety": {
                    "type": "string",
                    "enum": varieties.tolist()
                }
            },
            "additionalProperties": False,
            "required": ["variety"],
        },
        "strict": True
    }
}

To distill a model, you need to store all completions from a model, allowing you to give it as a reference to the smaller model to fine-tune it. We're therefore adding a store=True parameter to our client.chat.completions.create method so we can store those completions from gpt-4o.

We're going to store all completions (even 4o-mini and our future fine-tuned model) so we are able to run Evals from OpenAI platform directly.

When storing those completions, it's useful to store them with a metadata tag, that will allow filtering from the OpenAI platform to run distillation & evals on the specific set of completions you'd like to run those.

In [ ]:
# Initialize the progress index
metadata_value = "wine-distillation" # that's a funny metadata tag :-)

# Function to call the API and process the result for a single model (blocking call in this case)
def call_model(model, prompt):
    response = client.chat.completions.create(
        model=model,
        store=True,
        metadata={
            "distillation": metadata_value,
        },
        messages=[
            {
                "role": "system",
                "content": "You're a sommelier expert and you know everything about wine. You answer precisely with the name of the variety/blend."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
         response_format=response_format
    )
    return json.loads(response.choices[0].message.content.strip())['variety']

**Parallel processing**

As we'll run this on a large number of rows, let's make sure we run those completions in parallel and use concurrent futures for this. We'll iterate on our dataframe and output progress every 20 rows. We'll store the completion from the model we run the completion for in the same dataframe using the column name {model}-variety.

In [ ]:
def process_example(index, row, model, df, progress_bar):
    global progress_index

    try:
        # Generate the prompt using the row
        prompt = generate_prompt(row, varieties)

        df.at[index, model + "-variety"] = call_model(model, prompt)

        # Update the progress bar
        progress_bar.update(1)

        progress_index += 1
    except Exception as e:
        print(f"Error processing model {model}: {str(e)}")

def process_dataframe(df, model):
    global progress_index
    progress_index = 1  # Reset progress index

    # Create a tqdm progress bar
    with tqdm(total=len(df), desc="Processing rows") as progress_bar:
        # Process each example concurrently using ThreadPoolExecutor
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(process_example, index, row, model, df, progress_bar): index for index, row in df.iterrows()}

            for future in concurrent.futures.as_completed(futures):
                try:
                    future.result()  # Wait for each example to be processed
                except Exception as e:
                    print(f"Error processing example: {str(e)}")

    return df

Let's try out our call model function before processing the whole dataframe and check the output.



In [ ]:
answer = call_model('gpt-4o', generate_prompt(df_france_subset.iloc[2], varieties))
answer

'Pinot Noir'

In [ ]:
df_france_subset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
28068,France,"Good structured fruit, with ripe sweet cherry ...",Vieilles Vignes,88,15.0,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Château du Châtelard 2007 Vieilles Vignes (Be...,Gamay,Château du Châtelard
47796,France,"This is a dry wine, packed with tannins at thi...",Cuvée Henri,88,29.0,Bordeaux,Blaye Côtes de Bordeaux,NaN,Roger Voss,@vossroger,Château Berthenon 2014 Cuvée Henri (Blaye Côt...,Bordeaux-style Red Blend,Château Berthenon
117692,France,"Densely tannic, this is a wine that shows some...",NaN,87,20.0,Bordeaux,Médoc,NaN,Roger Voss,@vossroger,Château Patache d'Aux 2009 Médoc,Bordeaux-style Red Blend,Château Patache d'Aux
119136,France,This is a direct but tasty and well-balanced P...,NaN,87,7.0,France Other,Vin de France,NaN,Lauren Buzzeo,@laurbuzz,Le Pépin 2015 Pinot Noir (Vin de France),Pinot Noir,Le Pépin
41574,France,A strangely tart notion persists on the nose. ...,Les Natures,86,25.0,Alsace,Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Jean-Baptiste Adam 2014 Les Natures Pinot Noir...,Pinot Noir,Jean-Baptiste Adam


Great! We confirmed we can get a grape variety as an output, let's now process the dataset with both gpt-4o and gpt-4o-mini and compare the results.



In [ ]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o")

Processing rows: 100%|██████████| 500/500 [01:13<00:00,  6.77it/s]


In [ ]:
df_france_subset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,gpt-4o-variety
28068,France,"Good structured fruit, with ripe sweet cherry ...",Vieilles Vignes,88,15.0,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Château du Châtelard 2007 Vieilles Vignes (Be...,Gamay,Château du Châtelard,Gamay
47796,France,"This is a dry wine, packed with tannins at thi...",Cuvée Henri,88,29.0,Bordeaux,Blaye Côtes de Bordeaux,NaN,Roger Voss,@vossroger,Château Berthenon 2014 Cuvée Henri (Blaye Côt...,Bordeaux-style Red Blend,Château Berthenon,Merlot-Cabernet Sauvignon
117692,France,"Densely tannic, this is a wine that shows some...",NaN,87,20.0,Bordeaux,Médoc,NaN,Roger Voss,@vossroger,Château Patache d'Aux 2009 Médoc,Bordeaux-style Red Blend,Château Patache d'Aux,Bordeaux-style Red Blend
119136,France,This is a direct but tasty and well-balanced P...,NaN,87,7.0,France Other,Vin de France,NaN,Lauren Buzzeo,@laurbuzz,Le Pépin 2015 Pinot Noir (Vin de France),Pinot Noir,Le Pépin,Pinot Noir
41574,France,A strangely tart notion persists on the nose. ...,Les Natures,86,25.0,Alsace,Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Jean-Baptiste Adam 2014 Les Natures Pinot Noir...,Pinot Noir,Jean-Baptiste Adam,Sylvaner


In [ ]:
df_france_subset = process_dataframe(df_france_subset, "gpt-4o-mini")

Processing rows: 100%|██████████| 500/500 [00:53<00:00,  9.39it/s]


In [ ]:
df_france_subset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,gpt-4o-variety,gpt-4o-mini-variety
28068,France,"Good structured fruit, with ripe sweet cherry ...",Vieilles Vignes,88,15.0,Beaujolais,Beaujolais-Villages,NaN,Roger Voss,@vossroger,Château du Châtelard 2007 Vieilles Vignes (Be...,Gamay,Château du Châtelard,Gamay,Gamay
47796,France,"This is a dry wine, packed with tannins at thi...",Cuvée Henri,88,29.0,Bordeaux,Blaye Côtes de Bordeaux,NaN,Roger Voss,@vossroger,Château Berthenon 2014 Cuvée Henri (Blaye Côt...,Bordeaux-style Red Blend,Château Berthenon,Merlot-Cabernet Sauvignon,Bordeaux-style Red Blend
117692,France,"Densely tannic, this is a wine that shows some...",NaN,87,20.0,Bordeaux,Médoc,NaN,Roger Voss,@vossroger,Château Patache d'Aux 2009 Médoc,Bordeaux-style Red Blend,Château Patache d'Aux,Bordeaux-style Red Blend,Cabernet Sauvignon
119136,France,This is a direct but tasty and well-balanced P...,NaN,87,7.0,France Other,Vin de France,NaN,Lauren Buzzeo,@laurbuzz,Le Pépin 2015 Pinot Noir (Vin de France),Pinot Noir,Le Pépin,Pinot Noir,Pinot Noir
41574,France,A strangely tart notion persists on the nose. ...,Les Natures,86,25.0,Alsace,Alsace,NaN,Anne Krebiehl MW,@AnneInVino,Jean-Baptiste Adam 2014 Les Natures Pinot Noir...,Pinot Noir,Jean-Baptiste Adam,Sylvaner,Pinot Blanc


**Comparing gpt-4o and gpt-4o-mini**

Now that we've got all chat completions for those two models ; let's compare them against the expected grape variety and assess their accuracy at finding it. We'll do this directly in python here as we've got a simple string check to run, but if your task involves more complex evals you can leverage OpenAI Evals or our open-source eval framework.

In [ ]:
models = ['gpt-4o', 'gpt-4o-mini']


In [ ]:

def get_accuracy(model, df):
    return np.mean(df['variety'] == df[model + '-variety'])

In [ ]:


for model in models:
    print(f"{model} accuracy: {get_accuracy(model, df_france_subset) * 100:.2f}%")

gpt-4o accuracy: 81.00%
gpt-4o-mini accuracy: 65.80%


We can see that gpt-4o is better a finding grape variety than 4o-mini (12.80% higher or almost 20% relatively to 4o-mini!). Now I'm wondering if we're making gpt-4o drink wine during training!



**Distilling gpt-4o outputs to gpt-4o-mini**

Let's assume we'd like to run this prediction often, we want completions to be faster and cheaper, but keep that level of accuracy. That'd be great to be able to distill 4o accuracy to 4o-mini, wouldn't it? Let's do it!

We'll now go to OpenAI Stored completions page: https://platform.openai.com/chat-completions.

Let's select the model gpt-4o (make sure to do this, you don't want to distill the outputs of 4o-mini that we ran). Let's also select the metadata distillation: wine-distillation to get only stored completions ran from this cookbook.

Once you've selected completions, you can click on "Distill" on the top right corner to fine-tune a model based on those completions. Once we've done that, a file to run the fine-tuning process will automatically be created. Let's then select gpt-4o-mini as the base model, keep the default parameters (but you're free to change them or iterate with it to improve performance).



Once the fine-tuning job is starting, you can retrieve the fine tuning job ID from the fine-tuning page, we'll use it to monitor status of the fine-tuned job as well as retrieving the fine-tuned model id once done.



In [ ]:
# copy paste your fine-tune job ID below
import time
while True:

    finetune_job = client.fine_tuning.jobs.retrieve("ftjob-1lQuu5zrJ1rwSp9ozZEgpBKZ")

    if finetune_job.status == 'succeeded':
        fine_tuned_model = finetune_job.fine_tuned_model
        print('finetuned model: ' + fine_tuned_model)
        break
    else:
        print('finetuned job status: ' + finetune_job.status)
    time.sleep(20)

finetuned model: ft:gpt-4o-mini-2024-07-18:personal::AwANTJVV


**Running completions for the distilled model**


Now that we've got our model fine-tuned, we can use this model to run completions and compare accuracy with both gpt4o and gpt4o-mini. Let's grab a different subset of french wines (as we restricted the outputs to french grape varieties, without outliers, we'll need to focus our validation dataset to this too). Let's run this on 300 entries for each models.

In [ ]:
validation_dataset = df_france.sample(n=300)

models.append(fine_tuned_model)

for model in models:
    another_subset = process_dataframe(validation_dataset, model)

Processing rows: 100%|██████████| 300/300 [00:41<00:00,  7.26it/s]


**Let's compare accuracy of models**



In [ ]:
another_subset.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,gpt-4o-variety,gpt-4o-mini-variety,ft:gpt-4o-mini-2024-07-18:personal::AwANTJVV-variety
14143,France,"Bright and fruity, it has good acidity and a c...",Les Amandiers,87,NaN,Burgundy,Mâcon-Villages,NaN,Roger Voss,@vossroger,Domaine Sophie Cinier 2012 Les Amandiers (Mâc...,Chardonnay,Domaine Sophie Cinier,Chardonnay,Chardonnay,Chardonnay
6433,France,"A smooth, dense wine, ripe and polished. It br...",NaN,95,NaN,Bordeaux,Pessac-Léognan,NaN,Roger Voss,@vossroger,Château Pape Clément 2010 Pessac-Léognan,Bordeaux-style Red Blend,Château Pape Clément,Bordeaux-style Red Blend,Bordeaux-style Red Blend,Bordeaux-style Red Blend
15137,France,This is one of a collection of Champagnes tast...,Special Club Brut,93,56.0,Champagne,Champagne,NaN,Roger Voss,@vossroger,Charlier & Fils 2005 Special Club Brut (Champ...,Champagne Blend,Charlier & Fils,Champagne Blend,Chardonnay,Champagne Blend
23559,France,Light pink in the glass with soft aromas and f...,NaN,84,16.0,Languedoc-Roussillon,Vin de Pays d'Oc,NaN,Lauren Buzzeo,@laurbuzz,Mas Belles Eaux 2008 Rosé (Vin de Pays d'Oc),Rosé,Mas Belles Eaux,Rosé,Rosé,Rosé
15076,France,"A very soft, fruity wine, with a gentle, cream...",Florières,84,10.0,Burgundy,Mâcon-Villages,NaN,Roger Voss,@vossroger,Cave de Lugny 2012 Florières (Mâcon-Villages),Chardonnay,Cave de Lugny,Chardonnay,Chardonnay,Chardonnay


In [ ]:
for model in models:
    print(f"{model} accuracy: {get_accuracy(model, another_subset) * 100:.2f}%")

gpt-4o accuracy: 85.67%
gpt-4o-mini accuracy: 66.67%
ft:gpt-4o-mini-2024-07-18:personal::AwANTJVV accuracy: 80.67%


Our fine-tuned model performs way better than gpt-4o-mini, while having the same base model. We'll be able to use this model to run inferences at a lower cost and lower latency for future grape variety prediction.



***That's almost a 15% relative improvement over the non-distilled gpt-4o-mini!*** 🎉